# Wind River, Wyoming USGS Stream Gauge Time Series Analysis 
@millie-spencer September 2023 

In [1]:
# importing standard library packages first
import subprocess
from io import BytesIO

# import third party packages (alphabetized)
import folium
import hvplot.pandas
import pandas as pd
import requests


#supress warnings 
import warnings 
warnings.filterwarnings("ignore")

KeyboardInterrupt: 


### USGS Stream Gauge Dubois, Wyoming

![Wind River near Dubois Wyoming.](https://windriver.org/wp-content/uploads/2021/06/bill-sincavage-fly-fishing-brooks-lake-dubois-1200x645.jpg)

> Image Source: [Wind River Country](https://windriver.org/wp-content/uploads/2021/06/bill-sincavage-fly-fishing-brooks-lake-dubois-1200x645.jpg)

In [ ]:
##Site Map
sg_lat = 43.57856415
sg_lon = -109.7598835
# Initialize map and tweak settings
m = folium.Map(
    # Location to display - center at lat,lon
    location=(sg_lat, sg_lon),
    # Turns off annoying zooming while trying to scroll to the next cell
    scrollWheelZoom=False)

# Put a marker at the stream gauge location
folium.Marker([sg_lat, sg_lon], popup="Dubois Stream Gauge").add_to(m)

# Display the map
m

The data linked above contains daily average streamflow, measured in cubic feet per second [cfs], for a USGS streamgauge near Dubois, WY on the Wind River. 
Citation: U.S. Geological Survey Streamflow Data, Retrieved on September 19, 2023 from https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no=06218500&legacy=&referred_module=sw&period=&begin_date=1945-10-01&end_date=2022-09-30

&#9998; USGS streamflow URL: https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no=06218500&legacy=&referred_module=sw&period=&begin_date=1945-10-01&end_date=2022-09-30

In [ ]:
#Downloading gauge data
import requests

nwis_url = (
    "https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no"
    "=06218500&legacy=&referred_module=sw&period=&begin_date=1945-10-01"
    "&end_date=2022-09-30")


# Send an HTTP GET request to the URL
nwis_response = requests.get(nwis_url)
nwis_response.raise_for_status()
nwis_response

<Response [200]>

In [ ]:
#Cleaning data 
dubois_q_df = pd.read_csv(
    BytesIO(nwis_response.content),
    comment="#",
    delimiter="\t",
    skiprows=[28, 29],
    names=["agency_cd", "site_no", "datetime", "streamflow_cfs", "code"],
    index_col="datetime",
    parse_dates=True,
    na_values='Ice'
)
dubois_q_df

,agency_cd,site_no,streamflow_cfs,code
datetime,,,,
1945-10-01,USGS,6218500,105.0,A
1945-10-02,USGS,6218500,105.0,A
1945-10-03,USGS,6218500,105.0,A
1945-10-04,USGS,6218500,105.0,A
1945-10-05,USGS,6218500,105.0,A
...,...,...,...,...
2022-09-26,USGS,6218500,79.8,A
2022-09-27,USGS,6218500,77.9,A
2022-09-28,USGS,6218500,76.6,A


In [ ]:
#trying to observe the June 2, 1956 flood in our data 
dubois_flood_df = dubois_q_df['1955-10':'1956-09']
dubois_flood_df

,agency_cd,site_no,streamflow_cfs,code
datetime,,,,
1955-10-01,USGS,6218500,70.0,A
1955-10-02,USGS,6218500,73.0,A
1955-10-03,USGS,6218500,71.0,A
1955-10-04,USGS,6218500,70.0,A
1955-10-05,USGS,6218500,67.0,A
...,...,...,...,...
1956-09-26,USGS,6218500,86.0,A
1956-09-27,USGS,6218500,86.0,A
1956-09-28,USGS,6218500,91.0,A


In [ ]:
#Plotting flood data 
dubois_flood_df.hvplot(y='streamflow_cfs', title='Streamflow [cfs] at '
                      'Dubois Gauge from Oct 1955-Sept 1956')

:Curve   [datetime]   (streamflow_cfs)

In [ ]:
#Resampling to annual maxiuma
dubois_ann_max_q_df = dubois_q_df[['streamflow_cfs']].resample('AS').max()
dubois_ann_max_q_df

,streamflow_cfs
datetime,
1945-01-01,105.0
1946-01-01,774.0
1947-01-01,904.0
1948-01-01,1170.0
1949-01-01,802.0
...,...
2018-01-01,1780.0
2019-01-01,1250.0
2020-01-01,1670.0


In [ ]:
#Plot resampled data 
dubois_ann_max_q_df.hvplot()

:Curve   [datetime]   (streamflow_cfs)

### Dubois, WY streamflow follows fairly consistent pattern of peaks and troughs in flood data since the 1950s

Despite significant flooding documented for June of 1956, an analysis of the complete timeseries indicates that this flood event does not stand out among consistent patterns of flooding on the Wind River near Dubois. 

In [ ]:
#Calculating exceedance probability and return period for 1956 flood
dubois_ann_max_q_post_df = dubois_ann_max_q_df['1956':].copy()
dubois_ann_max_q_post_df['exceed_prob'] = (
    dubois_ann_max_q_post_df.rank(
    ascending=False)
    / len(dubois_ann_max_q_post_df))

dubois_ann_max_q_post_df['return_period'] = (
    1 / dubois_ann_max_q_post_df.exceed_prob)
dubois_ann_max_q_post_df

,streamflow_cfs,exceed_prob,return_period
datetime,,,
1956-01-01,1790.0,0.074627,13.400000
1957-01-01,1460.0,0.179104,5.583333
1958-01-01,1010.0,0.462687,2.161290
1959-01-01,1100.0,0.402985,2.481481
1960-01-01,676.0,0.776119,1.288462
...,...,...,...
2018-01-01,1780.0,0.089552,11.166667
2019-01-01,1250.0,0.283582,3.526316
2020-01-01,1670.0,0.119403,8.375000


In [ ]:
#Plotting return period for 1956 flood 
dubois_ann_max_q_post_df.return_period.hvplot()

:Curve   [datetime]   (return_period)

The return period plot indicates that the return period of the 1972 flood would likely be 67 years. 

In [ ]:
%%capture
%%bash
jupyter nbconvert wind-river-usgs-time-series.ipynb --to html --no-input